<a href="https://colab.research.google.com/github/rsshnf/NPL-w-Python/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de dados utilizando SVM

In [0]:
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from string import punctuation
import unicodedata
import re

In [0]:
dados_textos = pd.read_csv("C:\\Users\\Raissa\\analise\\brutos\\classificados_textos.csv", sep=";", encoding='ISO-8859-1')

In [0]:
dados_textos.head()

,Voto,Text
0,1,Ainda tô sem palavras pra essa união e pro dia...
1,0,#AFederalLuta\n#TiraAMãoDoNossoIF https://t.co...
2,1,É incrível ver a quantidade de estudante que a...
3,0,#AFederalLuta\n#HojeAAulaÉNaRua https://t.co/q...
4,1,Um amor por essa Foto #AFEDERALLUTA \n#TIRAAMA...


In [0]:
def limpeza_dados(tuites, text_field):
    tuites[text_field] = tuites[text_field].str.lower()
    tuites[text_field] = tuites[text_field].str.replace(r"#", " ") #remove hashtags
    tuites[text_field] = tuites[text_field].str.replace(r"http", " ")
    tuites[text_field] = tuites[text_field].str.replace(r"http\S+", " ")
    tuites[text_field] = tuites[text_field].str.replace(r"@", "at")
    tuites[text_field] = tuites[text_field].str.replace(r"\n", " ") #remove as linhas em branco
    return tuites

In [0]:
X = dados_textos["Text"].values.astype('U')
y = dados_textos["Voto"].values.astype('U')

In [0]:
#limpeza dos dados
dados_textos_limpos = limpeza_dados(dados_textos, "Text")

In [0]:
stopwords = ['pra', 'pro','to', 'ta','de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'a', 'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'a', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta', 'eu', 'tambam', 'sa3', 'pelo', 'pela', 'ata', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estao', 'vocaa', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'a s', 'minha', 'taam', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'sera', 'na3s', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocaas', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 
'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estivaramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivassemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'ha', 'havemos', 'hao', 'houve', 'houvemos', 'houveram', 'houvera', 'houvaramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvassemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houvera', 'houveremos', 'houverao', 'houveria', 'houveraamos', 'houveriam', 'sou', 'somos', 'sao', 'era', 'aramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fa ramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fa ssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'sera', 'seremos', 'serao', 'seria', 'seraamos', 'seriam', 'tenho', 'tem', 'temos', 'tam', 'tinha', 'tanhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivaramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivassemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'tera', 'teremos', 'terao', 'teria', 'teraamos', 'teriam']
print(len(stopwords))

224


In [0]:
def remove_stopWords(sentence):
    frase = []
    for word in sentence.split():
        if word not in stopwords:
           # semStop = [p for p in word.split() if p not in stopwords]
            frase.append(word)
    return ' '.join(frase)

In [0]:
dados_textos['Text'] = [remove_stopWords(str(t)) for t in dados_textos['Text']]

In [0]:
def removerAcentosECaracteresEspeciais(palavra):

    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    palavraSemAcento = re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

    return palavraSemAcento

In [0]:
dados_textos['Text'] = [removerAcentosECaracteresEspeciais(str(t)) for t in dados_textos['Text']]

In [0]:
#regravação do arquivo com os dados limpos 
dados_textos_limpos.to_csv("dados_limpos.csv")

In [0]:
dados_textos_limpos.head()

,Voto,Text
0,1,ainda to palavras uniao dia hoje lindo ifc amo...
1,0,afederalluta tiraamaodonossoif stcohpn35xub6d
2,1,e incrivel ver quantidade estudante ainda preo...
3,0,afederalluta hojeaaulaenarua stcoqazbuks6lc
4,1,amor foto afederalluta tiraamaodomeuif stcojbj...


In [0]:
dados_textos_limpos.tail()

,Voto,Text
4994,0,dia13erua tsunami13a stco9oib3kso6a
4995,0,enos tsunami13a dia13erua asruaspovo lulalivre...
4996,1,arrumando ir derrubar governo tsunami13a
4997,0,dce ufmg ate presente momento nao divulgou ond...
4998,1,recife ficou pequena tantos jovens rua defende...


In [0]:
vectorizer = CountVectorizer(analyzer = "word")
X_vetor = vectorizer.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_vetor, y, test_size = 0.3)

In [0]:
C = 1.0  # SVM regularization parameter
classificador = svm.SVC(kernel='linear', C=C)

In [0]:
#treinando o modelo
classificador.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
classificador.predict(X_test)

array(['-1', '0', '0', ..., '1', '0', '1'], dtype='<U21')

In [0]:
y_pred = classificador.predict(X_test)

In [0]:
classificador.score(X_test,y_test)

0.7193333333333334

In [0]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.42      0.29      0.34       189
           0       0.78      0.84      0.81       811
           1       0.69      0.69      0.69       500

    accuracy                           0.72      1500
   macro avg       0.63      0.61      0.61      1500
weighted avg       0.70      0.72      0.71      1500

